In [1]:
import heapq
import math

In [3]:
# Define the grid size
grid_size = 8

In [5]:
# Define the barriers with a high movement cost
barriers = [(2, 4), (2, 5), (2, 6), (3, 6), (4, 6), (5, 6), (5, 5), (5, 4), (5, 3), (5, 2), (4, 2), (3, 2)]
barrier_cost = 100

In [7]:
# Define start and goal positions
start = (0, 0)
goal = (7, 7)

In [9]:
# Possible movements (including diagonals)
movements = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]

In [11]:
def heuristic(a, b):
    """Calculate the Euclidean distance heuristic."""
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [30]:
def a_star_search(start, goal):
    """A* algorithm implementation."""
    open_list = []
    heapq.heappush(open_list, (0, start))  # Priority queue with (f, position)
    came_from = {}
    g_score = {start: 0}
    
    while open_list:
        _, current = heapq.heappop(open_list)
        
        # If we reached the goal, reconstruct the path
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path, g_score[goal]
        
        # Explore neighbors
        for move in movements:
            neighbor = (current[0] + move[0], current[1] + move[1])
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size:
                # Calculate movement cost
                tentative_g_score = g_score[current] + 1
                if neighbor in barriers:
                    tentative_g_score += barrier_cost

                # If this path to the neighbor is better, record it
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score, neighbor))
    
    return None, float('inf')  # No path found

def draw_grid(path):
    """Draw the grid with the path and barriers."""
    for i in range(grid_size):
        for j in range(grid_size):
            if (i, j) in barriers:
                print("█", end=" ")
            elif (i, j) == start:
                print("S", end=" ")  # Start
            elif (i, j) == goal:
                print("G", end=" ")  # Goal
            elif (i, j) in path:
                print("x", end=" ")  # Path
            else:
                print(".", end=" ")
        print()

if __name__ == "__main__":
    path, cost = a_star_search(start, goal)
    if path:
        print(f"Path found with cost {cost}: {path}")
        draw_grid(path)
    else:
        print("No path found.")

Path found with cost 11: [(1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
S . . . . . . . 
. x . . . . . . 
. . x . █ █ █ . 
. x █ . . . █ . 
. x █ . . . █ . 
. x █ █ █ █ █ . 
. . x . . . . . 
. . . x x x x G 


# pygame application

In [125]:
import pygame
import math
import heapq

In [127]:
# Initialize pygame
pygame.init()

(5, 0)

In [129]:
# Screen dimensions
width = 800
rows = 50  # More rows for finer grid
window = pygame.display.set_mode((width, width))
pygame.display.set_caption("A* Path Finding Algorithm (DSA Project)")

In [131]:
# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
PURPLE = (128, 0, 128)
ORANGE = (255, 165, 0)
GRAY = (128, 128, 128)

In [133]:
# Define grid size and barriers
grid_size = 50
barriers = [(10, 10), (10, 11), (10, 12), (11, 12), (12, 12), (13, 12), (14, 12), (15, 12), (16, 12), (17, 12), (18, 12)]
start = (5, 5)
goal = (45, 45)

In [135]:
# Node class representing each square in the grid
class Node:
    def __init__(self, row, col, width):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.f_score = float('inf')  # Add f_score attribute for comparison

    def get_pos(self):
        return self.row, self.col

    def is_closed(self):
        return self.color == RED

    def is_open(self):
        return self.color == GREEN

    def is_barrier(self):
        return self.color == BLACK

    def is_start(self):
        return self.color == ORANGE

    def is_goal(self):
        return self.color == BLUE

    def reset(self):
        self.color = WHITE

    def make_start(self):
        self.color = ORANGE

    def make_goal(self):
        self.color = BLUE

    def make_barrier(self):
        self.color = BLACK

    def make_closed(self):
        self.color = RED

    def make_open(self):
        self.color = GREEN

    def make_path(self):
        self.color = PURPLE

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        self.neighbors = []
        # Down
        if self.row < grid_size - 1 and not grid[self.row + 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col])
        # Up
        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col])
        # Right
        if self.col < grid_size - 1 and not grid[self.row][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col + 1])
        # Left
        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col - 1])

        # Diagonals
        if self.row < grid_size - 1 and self.col < grid_size - 1 and not grid[self.row + 1][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col + 1])
        if self.row > 0 and self.col > 0 and not grid[self.row - 1][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col - 1])
        if self.row < grid_size - 1 and self.col > 0 and not grid[self.row + 1][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col - 1])
        if self.row > 0 and self.col < grid_size - 1 and not grid[self.row - 1][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col + 1])

    def __lt__(self, other):
        # This is required for the priority queue (heapq) to compare Node objects based on their f_score
        return self.f_score < other.f_score

In [137]:
# Heuristic function (Euclidean Distance)
def heuristic(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [139]:
# A* algorithm implementation
def a_star_algorithm(draw, grid, start, goal):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {node: float('inf') for row in grid for node in row}
    g_score[start] = 0
    f_score = {node: float('inf') for row in grid for node in row}
    f_score[start] = heuristic(start.get_pos(), goal.get_pos())
    start.f_score = f_score[start]  # Set f_score for start node

    open_set_hash = {start}

    while open_set:
        current = heapq.heappop(open_set)[1]
        open_set_hash.remove(current)

        if current == goal:
            reconstruct_path(came_from, goal, draw)
            goal.make_goal()
            return True

        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1

            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor.get_pos(), goal.get_pos())
                neighbor.f_score = f_score[neighbor]  # Update f_score for neighbor node
                if neighbor not in open_set_hash:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        draw()

        if current != start:
            current.make_closed()

    return False

In [141]:
# Function to reconstruct the path
def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

In [143]:
# Create the grid
def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            node = Node(i, j, gap)
            grid[i].append(node)
    return grid

In [145]:
# Draw the grid lines
def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GRAY, (0, i * gap), (width, i * gap))
        pygame.draw.line(win, GRAY, (i * gap, 0), (i * gap, width))

In [147]:
# Draw the entire window
def draw(win, grid, rows, width):
    win.fill(WHITE)

    for row in grid:
        for node in row:
            node.draw(win)

    draw_grid(win, rows, width)
    pygame.display.update()

In [149]:
# Get the position of the mouse click in grid coordinates
def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos
    row = y // gap
    col = x // gap
    return row, col

def main(win, width):
    grid = make_grid(rows, width)

    start_node = None
    goal_node = None

    run = True
    started = False
     # Set barriers in the grid
    for barrier in barriers:
        row, col = barrier
        grid[row][col].make_barrier()

    while run:
        draw(win, grid, rows, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if started:
                continue

            if pygame.mouse.get_pressed()[0]:  # Left mouse button
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, rows, width)
                node = grid[row][col]
                if not start_node and node != goal_node:
                    start_node = node
                    start_node.make_start()

                elif not goal_node and node != start_node:
                    goal_node = node
                    goal_node.make_goal()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start_node and goal_node:
                    for row in grid:
                        for node in row:
                            node.update_neighbors(grid)

                    a_star_algorithm(lambda: draw(win, grid, rows, width), grid, start_node, goal_node)

    pygame.quit()

main(window, width)